In [ ]:
!pip install tensorflow==2.6.0


In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

labels = [
        {'name':'A', 'id':1},
        {'name':'B', 'id':2},
        {'name':'C', 'id':3},
        {'name':'D', 'id':4},
        {'name':'E', 'id':5}]

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'











import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections,prediction_dict
TT_cau_hoi=1







In [2]:
import cv2 
import numpy as np
import six
import time

category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

# Setup capture
cap = cv2.VideoCapture(1)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
SL = 0
TT_cau_hoi == 1
while True: 
    t = time.time()
    ret, frame = cap.read()
    image_np = np.array(frame)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections , prediction_dict  = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    def get_classes_name_and_scores(
        boxes,
        classes,
        scores,
        category_index,
        max_boxes_to_draw=20,
        min_score_thresh=.9): # returns bigger than 90% precision
        display_str = {}
        if not max_boxes_to_draw:
            max_boxes_to_draw = boxes.shape[0]
        for i in range(min(max_boxes_to_draw, boxes.shape[0])):
            if scores is None or scores[i] > min_score_thresh:
                if classes[i] in six.viewkeys(category_index):
                    display_str['name'] = category_index[classes[i]]['name']
                    display_str['score'] = '{}%'.format(int(100 * scores[i]))

        return display_str
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))


    result_predict = get_classes_name_and_scores(
      detections['detection_boxes'],
      detections['detection_classes']+label_id_offset,
      detections['detection_scores'],
      category_index)
    if (result_predict != {}):
        result_name = result_predict['name']
        result_score = result_predict['score'].rstrip("%")
        result_score = int(result_score)
    else:
        result_name=""
        result_score = 0
        
    if (result_name == "B") & (result_score > 70) & (TT_cau_hoi == 1)  :
        SL = SL+1
        if (SL == 10) & (TT_cau_hoi == 1):
            TT_cau_hoi = 2
            print("ban da tra loi dung chữ B")
            SL = 0
    if (result_name == "C") & (result_score > 95) & (TT_cau_hoi == 2)  :
        time.sleep(0.02)
        if (result_name == "C") & (result_score > 95) & (TT_cau_hoi == 2):
            time.sleep(0.02)
            if (result_name == "C") & (result_score > 95) & (TT_cau_hoi == 2):
                time.sleep(0.02)
                if (result_name == "C") & (result_score > 95) & (TT_cau_hoi == 2):
                    time.sleep(0.02)
                    if (result_name == "C") & (result_score > 95) & (TT_cau_hoi == 2):
                        TT_cau_hoi = 3
                        print("ban da tra loi dung chữ C")
    print(result_name,result_score)       
    print('fps', 1/(time.time()-t))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()

 0
fps 0.03650805897464251
 0
fps 17.416759405365003
 0
fps 18.187788093369353
 0
fps 17.27942521443226
 0
fps 14.965386577799027
 0
fps 17.10843530755425
 0
fps 18.899731439591932
 0
fps 18.915840981351614
 0
fps 17.971686148150688
 0
fps 17.580357030585258
 0
fps 18.612646274412352
 0
fps 17.502593484366066
 0
fps 20.454032965961183
 0
fps 20.04331412296547
 0
fps 15.910235449865906
 0
fps 18.59012498891942
 0
fps 17.590753151762723
 0
fps 20.053567928091606
 0
fps 18.56839793876503
 0
fps 18.21598756156245
 0
fps 17.589499066909898
 0
fps 18.220023196917502
 0
fps 17.590531829676944
 0
fps 17.57873604888496
 0
fps 16.99254553704544
 0
fps 18.560756183151383
 0
fps 19.640208468933352
 0
fps 16.977963439711143
 0
fps 17.901731143510773
 0
fps 17.904711492079212
C 98
fps 17.577557340843278
C 98
fps 17.895009898286574
C 92
fps 17.57895707424203
C 95
fps 17.699501630987495
C 97
fps 17.897835261385893
C 98
fps 18.387360373156575
C 98
fps 17.287615561848
C 98
fps 18.22928817920256
C 98
fps

 0
fps 19.274496918785527
 0
fps 19.66184453549095
 0
fps 18.262863412913703
 0
fps 20.053951451343767
 0
fps 19.28069909303619
 0
fps 19.59918506196146
 0
fps 19.64591208225017
 0
fps 19.665624223442315
 0
fps 19.65954056068283
 0
fps 19.6601856191994
 0
fps 19.272282823455892
 0
fps 20.12573594683429
 0
fps 18.799770510613882
 0
fps 19.28025594822197
 0
fps 19.15434709143551
 0
fps 17.985403463032682
 0
fps 21.3238974046112
 0
fps 20.025610297593186
 0
fps 19.24151535447881
 0
fps 20.89296246114609
 0
fps 19.28229458305175
 0
fps 20.05663652491596
 0
fps 18.957991701394853
 0
fps 19.12168791144665
 0
fps 16.819061825822647
 0
fps 17.892643388861636
 0
fps 18.635553383391834
 0
fps 18.14671206621296
 0
fps 18.136589668860427
 0
fps 18.241894166409338
 0
fps 17.906240260932304
 0
fps 17.906240260932304
 0
fps 17.906240260932304
 0
fps 18.26699940333869
 0
fps 16.953190113376852
 0
fps 16.72990383196321
 0
fps 18.059202679836215
 0
fps 19.653000215539457
 0
fps 19.548761162586924
 0
fps

 0
fps 19.242751229538282
 0
fps 18.21132111517804
 0
fps 20.06056953731072
 0
fps 18.349151074226867
 0
fps 18.9165234725586
 0
fps 19.659724858796785
 0
fps 19.280167321703555
 0
fps 18.56897337931706
 0
fps 19.87972547681341
 0
fps 19.42625828474033
 0
fps 20.063160412331683
 0
fps 19.656039552920777
 0
fps 19.286018420169302
 0
fps 20.004693178674653
 0
fps 19.659909160366173
 0
fps 20.033166481983876
 0
fps 17.888827756808052
 0
fps 18.281490134202738
 0
fps 20.02819214974692
 0
fps 18.542949857202224
 0
fps 18.217332574694773
 0
fps 19.471169067503517
 0
fps 21.328343181425243
 0
fps 21.78564973042602
 0
fps 19.94808357232202
 0
fps 19.616050883921055
 0
fps 20.01032408268808
 0
fps 19.571565891762248
 0
fps 19.94789382820563
 0
fps 19.287171327931723
 0
fps 20.05193811791253
 0
fps 19.625596586123642
 0
fps 19.285575030806864
 0
fps 19.66184453549095
 0
fps 20.88900841675382
 0
fps 18.631000555247084
 0
fps 18.915670366244694
 0
fps 18.909104024092258
 0
fps 21.28915418037114
 0

 0
fps 20.043601469948722
 0
fps 18.19812565081569
 0
fps 18.545491526023266
 0
fps 19.28061046244369
 0
fps 19.55714712026261
 0
fps 19.282826471737582
 0
fps 18.22968432856255
 0
fps 18.567986931634564
 0
fps 19.6604620858079
 0
fps 19.64103620731639
 0
fps 19.283890337144776
 0
fps 18.896410663038434
 0
fps 18.589136292724437
 0
fps 19.658895544494126
 0
fps 19.25220209215968
 0
fps 20.398426215476196
 0
fps 18.916267532584676
 0
fps 19.645728043016998
 0
fps 19.283801677210533
 0
fps 20.467807263251384
 0
fps 19.282117293343692
 0
fps 19.649501536616444
 0
fps 19.283535702299236
 0
fps 19.850371043465092
 0
fps 18.911150187113936
 0
fps 19.809588581738147
 0
fps 18.251181410730602
 0
fps 19.650974512743627
 0
fps 19.27307983917289
 0
fps 19.788373168269185
 0
fps 19.63450816640842
 0
fps 20.05366380752938
 0
fps 18.22160628717151
 0
fps 19.410345880806716
 0
fps 19.260247049639528
 0
fps 19.833100056742953
 0
fps 21.323138554768125
 0
fps 18.230714397482483
 0
fps 19.64628017106107

 0
fps 17.899210084966008
 0
fps 17.87152523307143
 0
fps 18.236025060760607
 0
fps 19.27520553673926
 0
fps 19.644071845069433
 0
fps 19.27963557970315
 0
fps 18.514383582807678
 0
fps 19.938600786267415
 0
fps 19.28371301809154
 0
fps 19.6527239587482
 0
fps 20.459620300091707
 0
fps 20.02293352428691
 0
fps 19.652171468464626
 0
fps 20.060665483712054
 0
fps 18.144513996738205
 0
fps 19.65871126192842
 0
fps 18.916950054572844
 0
fps 20.03680313380786
 0
fps 19.28087635666576
 0
fps 20.034410451147572
 0
fps 19.259008926275577
 0
fps 20.05232157882659
 0
fps 19.63221059332347
 0
fps 19.618987033884036
 0
fps 18.91891257966883
 0
fps 18.530170090567704
 0
fps 19.20284221755234
 0
fps 21.520618583149048
 0
fps 19.579148830891175
 0
fps 20.040249601039687
 0
fps 19.585457194355463
 0
fps 19.292671707972254
 0
fps 19.66064640119999
 0
fps 18.200652641810734
 0
fps 18.909274520765333
 0
fps 19.282383229128357
 0
fps 19.263696947150382
 0
fps 19.273611219608583
 0
fps 19.6604620858079
 0


 0
fps 20.053472049570654
 0
fps 20.072377834886268
 0
fps 20.0071742033963
 0
fps 18.189760002775536
 0
fps 19.65161900924412
 0
fps 19.63882905998914
 0
fps 19.63974864442176
 0
fps 20.4335057924839
 0
fps 18.204681484567942
 0
fps 20.053855569155445
 0
fps 20.009178557287267
 0
fps 19.649041276860878
 0
fps 18.91345264990102
 0
fps 18.546475584896818
 0
fps 19.65207938977079
 0
fps 18.221922938235025
 0
fps 19.282205937790202
 0
fps 21.432862362349574
 0
fps 19.662766276627664
 0
fps 19.273965489511294
 0
fps 18.513402926441703
 0
fps 19.653921127605336
 0
fps 19.61953765985911
 0
fps 19.97240065712721
 0
fps 19.62027187590633
 0
fps 19.961944649358685
 0
fps 20.200273555645456
 0
fps 20.010037689041553
 0
fps 20.07026476091128
 0
fps 19.64729248641559
 0
fps 19.640300436419487
 0
fps 18.394698640890812
 0
fps 19.261043070154894
 0
fps 19.092181148634193
 0
fps 19.551768341856118
 0
fps 19.886040480378536
 0
fps 17.890659057076196
 0
fps 19.26413933108586
 0
fps 20.446754057123915
 

ValueError: in user code:

    File "C:\Users\Admin\AppData\Local\Temp\ipykernel_19140\2010066187.py", line 59, in detect_fn  *
        image, shapes = detection_model.preprocess(image)
    File "C:\Users\Admin\anaconda3\envs\combine_env6\lib\site-packages\object_detection\meta_architectures\ssd_meta_arch.py", line 484, in preprocess  *
        normalized_inputs, self._image_resizer_fn)
    File "C:\Users\Admin\anaconda3\envs\combine_env6\lib\site-packages\object_detection\utils\shape_utils.py", line 492, in resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    File "C:\Users\Admin\anaconda3\envs\combine_env6\lib\site-packages\object_detection\utils\shape_utils.py", line 246, in static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    File "C:\Users\Admin\anaconda3\envs\combine_env6\lib\site-packages\object_detection\core\preprocessor.py", line 3327, in resize_image  *
        new_image = tf.image.resize_images(

    ValueErr

In [15]:
SL

46

In [14]:
result_predict

{}